In [1]:
import tqdm
import multiprocessing
import pandas as pd
import numpy as np
import scipy.stats

from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [2]:
def load_representation(multi_col_representation_vector_file_path):
    multi_col_representation_vector = pd.read_csv(multi_col_representation_vector_file_path)
    vals = multi_col_representation_vector.iloc[:,1:(len(multi_col_representation_vector.columns))]
    original_values_as_df = pd.DataFrame({'PDB_ID': pd.Series([], dtype='str'),'Vector': pd.Series([], dtype='object')})
    for index, row in tqdm.tqdm(vals.iterrows(), total = len(vals)):
        list_of_floats = [float(item) for item in list(row)]
        original_values_as_df.loc[index] = [multi_col_representation_vector.iloc[index]['PDB_ID']] + [list_of_floats]
    return original_values_as_df

In [3]:
ppi_affinity_file = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/generalized_representation_benchmark/DATA/auxilary_input/skempi_pipr/SKEMPI_all_dg_avg.txt"

In [4]:
ppi_affinity_df = pd.read_csv(ppi_affinity_file,sep="\t",header=None)

In [5]:
ppi_affinity_df.columns = ['Protein1', 'Protein2', 'Affinity']

In [6]:
ppi_affinity_df[ppi_affinity_df['Protein1'].str.contains("wt") & ppi_affinity_df['Protein2'].str.contains("wt")].to_csv("wild_type_interactions.csv",index=False)

In [7]:
len(set(ppi_affinity_df[ppi_affinity_df['Protein1'].str.contains("wt") & ppi_affinity_df['Protein2'].str.contains("wt")]['Protein1']))

158

In [8]:
ppi_affinity_df[ppi_affinity_df['Protein1'].str.contains("wt") & ppi_affinity_df['Protein2'].str.contains("wt")]['Protein1']

138      1A22_A_wt
182      1A4Y_A_wt
195      1ACB_E_wt
206     1AHW_AB_wt
223      1AK4_A_wt
           ...    
2609    3HFM_HL_wt
2660     3NPS_A_wt
2917     3SGB_E_wt
2940     3TGK_E_wt
2947     4CPA_A_wt
Name: Protein1, Length: 158, dtype: object

In [9]:
len(set(list(ppi_affinity_df['Protein1'])))

764

In [10]:
skempi_vectors_path = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/generalized_representation_benchmark\
/DATA/representation_vectors/skempi/"

In [11]:
seqvec_skempi_vectors_df = load_representation(skempi_vectors_path+"skempi_bert_avg_representation_multi_col.csv")

100%|██████████| 2882/2882 [00:05<00:00, 544.05it/s]


In [12]:
seqvec_skempi_vectors_df

,PDB_ID,Vector
0,1CSE_E_wt,"[0.4469349086284637, -0.3453298509120941, -0.1..."
1,1CSE_I_wt,"[0.4469349086284637, -0.3453298509120941, -0.1..."
2,1CSE_I_LI45G_,"[0.4469349086284637, -0.3453298509120941, -0.1..."
3,1CSE_I_LI45S_,"[0.4469349086284637, -0.3453298509120941, -0.1..."
4,1CSE_I_LI45P_,"[0.4469349086284637, -0.3453298509120941, -0.1..."
...,...,...
2877,1PPF_I_EI10D_AI15V_TI17S_EI19D_RI21M_KI29T_GI3...,"[0.4469349086284637, -0.3453298509120941, -0.1..."
2878,1PPF_I_EI10D_KI13R_AI15V_EI19D_RI21M_GI32S_EI4...,"[0.4469349086284637, -0.3453298509120941, -0.1..."
2879,1PPF_I_EI10D_AI15V_EI19D_RI21M_GI32S_EI43D_LI4...,"[0.4469349086284637, -0.3453298509120941, -0.1..."
2880,1PPF_I_EI10D_LI18F_RI21M_EI43D_,"[0.4469349086284637, -0.3453298509120941, -0.1..."


## Calculate vector element-wise multiplication as described in https://academic.oup.com/bioinformatics/article/35/14/i305/5529260 

In [13]:
'1CSE_I_LI45S_' in seqvec_skempi_vectors_df['PDB_ID']  

False

In [14]:
rep_prot_list = list(seqvec_skempi_vectors_df['PDB_ID'])

In [15]:
rep_prot_list

['1CSE_E_wt',
 '1CSE_I_wt',
 '1CSE_I_LI45G_',
 '1CSE_I_LI45S_',
 '1CSE_I_LI45P_',
 '1CSE_I_LI45I_',
 '1CSE_I_LI45D_',
 '1CSE_I_LI45E_',
 '1ACB_E_wt',
 '1ACB_I_wt',
 '1ACB_I_LI45G_',
 '1ACB_I_LI45S_',
 '1ACB_I_LI45P_',
 '1ACB_I_LI45I_',
 '1ACB_I_LI45D_',
 '1ACB_I_LI45E_',
 '1SBN_E_wt',
 '1SBN_I_wt',
 '1SBN_I_RI45K_',
 '1SIB_E_wt',
 '1SIB_I_wt',
 '1SIB_I_KI53R_',
 '1TM1_E_wt',
 '1TM1_I_wt',
 '1TM1_I_YI61A_',
 '1TM1_I_YI61G_',
 '1TM1_I_RI65A_',
 '1TM1_I_RI67A_',
 '1TM1_I_RI67C_',
 '1TM1_I_RI67A_RI65A_',
 '1TM1_I_TI58D_',
 '1TM1_I_TI58A_',
 '1TM1_I_EI60A_',
 '1TM1_I_TI58A_EI60A_',
 '1TM1_I_TI58D_EI60A_',
 '1TM1_I_VI70A_',
 '1Y1K_E_wt',
 '1Y1K_I_wt',
 '1Y1K_I_AI58T_',
 '1Y33_E_wt',
 '1Y33_I_wt',
 '1Y33_I_PI58T_',
 '1Y34_E_wt',
 '1Y34_I_wt',
 '1Y34_I_AI60E_',
 '1Y3B_E_wt',
 '1Y3B_I_wt',
 '1Y3B_I_SI60E_',
 '1Y4A_E_wt',
 '1Y4A_I_wt',
 '1Y4A_I_SI60E_RI59M_',
 '1Y3C_E_wt',
 '1Y3C_I_wt',
 '1Y3C_I_AI62R_',
 '1Y48_E_wt',
 '1Y48_I_wt',
 '1Y48_I_AI65R_',
 '1Y3D_E_wt',
 '1Y3D_I_wt',
 '1Y3D_I_AI67R_',


In [16]:
multiplied_vectors = pd.DataFrame({'Protein1': pd.Series([], dtype='str'),\
                                   'Protein2': pd.Series([], dtype='str'),\
                                   'Vector': pd.Series([], dtype='object')}) 

rep_prot_list = list(seqvec_skempi_vectors_df['PDB_ID'])
for index,row in tqdm.tqdm(ppi_affinity_df.iterrows()):
    if row['Protein1'] in rep_prot_list and row['Protein2'] in rep_prot_list:
        vec1 = list(seqvec_skempi_vectors_df[seqvec_skempi_vectors_df['PDB_ID']\
                                             == row['Protein1']]['Vector'])[0]
        vec2 = list(seqvec_skempi_vectors_df[seqvec_skempi_vectors_df['PDB_ID']\
                                             == row['Protein2']]['Vector'])[0]
        multiplied_vec = np.multiply(vec1,vec2)
        multiplied_vectors = multiplied_vectors.\
            append({'Protein1':row['Protein1'], 'Protein2':row['Protein2'],\
                    'Vector':multiplied_vec},ignore_index = True)
    

2950it [00:06, 434.19it/s]


In [17]:
multiplied_vectors

,Protein1,Protein2,Vector
0,1A22_A_wt,1A22_B_CB308A_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
1,1A22_A_wt,1A22_B_CB322A_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
2,1A22_A_wt,1A22_B_DB319A_EB320A_KB321A_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
3,1A22_A_wt,1A22_B_DB326A_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
4,1A22_A_wt,1A22_B_DB332A_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
...,...,...,...
2945,4CPA_A_wt,4CPA_I_VI38I_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
2946,4CPA_A_wt,4CPA_I_VI38L_,"[0.19975081255073318, 0.11925270593096915, 0.0..."
2947,4CPA_A_wt,4CPA_I_wt,"[0.19975081255073318, 0.11925270593096915, 0.0..."
2948,4CPA_A_wt,4CPA_I_YI37F_,"[0.19975081255073318, 0.11925270593096915, 0.0..."


In [18]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    mae = mean_absolute_error(y_train, predictions)
    corr = scipy.stats.pearsonr(y_train, predictions)
    return mse,mae,corr
    
def calc_validation_error(X_test, y_test, model):
    '''returns out-of-sample error for already fit model.'''
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    corr = scipy.stats.pearsonr(y_test, predictions)
    return mse,mae,corr
    
def calc_metrics(X_train, y_train, X_test, y_test, model):
    '''fits model and returns the metrics for in-sample error and out-of-sample error'''
    model.fit(X_train, y_train)
    train_mse_error,train_mae_error,train_corr = calc_train_error(X_train, y_train, model)
    val_mse_error,val_mae_error,val_corr = calc_validation_error(X_test, y_test, model)
    return train_mse_error, val_mse_error, train_mae_error, val_mae_error,train_corr,val_corr

In [ ]:
y_train = [0.45462982, 0.45462982, 0.4391372 , 0.5315291 , 0.46772246,0.49276013]



In [19]:
l = [1,2,3]


In [20]:
def report_results(
    train_mse_error_list,
    validation_mse_error_list,
    train_mae_error_list,
    validation_mae_error_list,
    train_corr_list,
    validation_corr_list,
    train_corr_pval_list,
    validation_corr_pval_list,
):
    result_df = pd.DataFrame(
        {
            "train_mse_error": round(np.mean(train_mse_error_list) * 100, 4),
            "train_mse_std": round(np.std(train_mse_error_list) * 100, 4),
            "val_mse_error": round(np.mean(validation_mse_error_list) * 100, 4),
            "val_mse_std": round(np.std(validation_mse_error_list) * 100, 4),
            "train_mae_error": round(np.mean(train_mae_error_list) * 100, 4),
            "train_mae_std": round(np.std(train_mae_error_list) * 100, 4),
            "val_mae_error": round(np.mean(validation_mae_error_list) * 100, 4),
            "val_mae_std": round(np.std(validation_mae_error_list) * 100, 4),
            "train_corr": round(np.mean(train_corr_list), 4),
            "train_corr_pval": round(np.mean(train_corr_pval_list), 4),
            "validation_corr": round(np.mean(validation_corr_list), 4),
            "validation_corr_pval": round(np.mean(validation_corr_pval_list), 4),
        },
        index=[0],
    )

    result_detail_df = pd.DataFrame(
        {
            "train_mse_errors": list(np.multiply(train_mse_error_list, 100)),
            "val_mse_errors": list(np.multiply(validation_mse_error_list, 100)),
            "train_mae_errors": list(np.multiply(train_mae_error_list, 100)),
            "val_mae_errors": list(np.multiply(validation_mae_error_list, 100)),
            "train_corrs": list(np.multiply(train_corr_list, 100)),
            "train_corr_pvals": list(np.multiply(train_corr_pval_list, 100)),
            "validation_corr": list(np.multiply(validation_corr_list, 100)),
            "validation_corr_pval": list(np.multiply(validation_corr_pval_list, 100)),
        },
        index=range(len(train_mse_error_list)),
    )
    return result_df, result_detail_df


In [21]:
def predictAffinityWithModel(regressor_model):
    K = 10
    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    train_mse_error_list = []
    validation_mse_error_list = []
    train_mae_error_list = []
    validation_mae_error_list = []
    train_corr_list = []
    validation_corr_list = []
    train_corr_pval_list = []
    validation_corr_pval_list = []

    data = np.array(np.asarray(multiplied_vectors["Vector"].tolist()), dtype=float)
    ppi_affinity_filtered_df = ppi_affinity_df\
        [ppi_affinity_df['Protein1'].isin(multiplied_vectors['Protein1']) & \
         ppi_affinity_df['Protein2'].isin(multiplied_vectors['Protein2']) ]
    target = np.array(ppi_affinity_filtered_df["Affinity"])
    scaler = MinMaxScaler()
    scaler.fit(target.reshape(-1, 1))
    target = scaler.transform(target.reshape(-1, 1))[:, 0]
    for train_index, val_index in tqdm.tqdm(kf.split(data, target), total=K):

        # split data
        X_train, X_val = data[train_index], data[val_index]
        y_train, y_val = target[train_index], target[val_index]

        # instantiate model
        reg = regressor_model #linear_model.BayesianRidge()

        # calculate error_list
        (
            train_mse_error,
            val_mse_error,
            train_mae_error,
            val_mae_error,
            train_corr,
            val_corr,
        ) = calc_metrics(X_train, y_train, X_val, y_val, reg)

        # append to appropriate list
        train_mse_error_list.append(train_mse_error)
        validation_mse_error_list.append(val_mse_error)

        train_mae_error_list.append(train_mae_error)
        validation_mae_error_list.append(val_mae_error)

        train_corr_list.append(train_corr[0])
        validation_corr_list.append(val_corr[0])

        train_corr_pval_list.append(train_corr[1])
        validation_corr_pval_list.append(val_corr[1])

    return report_results(
        train_mse_error_list,
        validation_mse_error_list,
        train_mae_error_list,
        validation_mae_error_list,
        train_corr_list,
        validation_corr_list,
        train_corr_pval_list,
        validation_corr_pval_list,
    )

In [22]:
model = linear_model.BayesianRidge()
result_df, result_detail_df = predictAffinityWithModel(model)

  0%|          | 0/10 [00:00<?, ?it/s]/media/DATA/serbulent/conda_envs/skempi/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/media/DATA/serbulent/conda_envs/skempi/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
 10%|█         | 1/10 [00:01<00:09,  1.10s/it]/media/DATA/serbulent/conda_envs/skempi/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/media/DATA/serbulent/conda_envs/skempi/lib/python3.9/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is n

In [23]:
result_df

,train_mse_error,train_mse_std,val_mse_error,val_mse_std,train_mae_error,train_mae_std,val_mae_error,val_mae_std,train_corr,train_corr_pval,validation_corr,validation_corr_pval
0,2.3458,0.0115,2.3474,0.1029,12.027,0.0415,12.0305,0.3593,NaN,NaN,NaN,NaN


In [61]:
result_detail_df

,train_mse_errors,val_mse_errors,train_mae_errors,val_mae_errors,train_corrs,train_corr_pvals,validation_corr,validation_corr_pval
0,1.757003,1.756543,10.130397,10.231082,49.382482,3.110485e-160,45.485794,2.031100e-14
1,1.746865,1.877006,10.123038,10.339209,48.563335,3.480839e-154,51.844779,1.260535e-19
2,1.756751,1.756783,10.155731,9.883432,49.078154,5.005101e-158,49.056762,3.785524e-17
3,1.737997,1.938557,10.065078,10.739856,49.042301,9.226276e-158,48.403158,1.293217e-16
4,1.762882,1.714857,10.171091,9.905500,49.294463,1.230775e-159,45.819260,1.296181e-14
5,1.784332,1.498729,10.212101,9.479030,48.722592,2.087392e-155,53.640803,3.133354e-21
6,1.759358,1.738988,10.160627,10.167630,48.363621,8.584089e-153,54.314889,6.946794e-22
7,1.732155,1.984052,10.065873,10.826505,50.110435,8.212809e-166,39.713834,1.648531e-10
8,1.751489,1.810791,10.117778,10.267877,48.990581,2.226540e-157,49.134132,3.267446e-17
9,1.749020,1.824198,10.107870,10.427031,50.002650,5.494180e-165,39.833445,1.394508e-10


In [62]:
model = RandomForestRegressor(n_estimators=100,n_jobs=multiprocessing.cpu_count(),random_state=42)
result_df, result_detail_df = predictAffinityWithModel(model)

100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


In [63]:
result_detail_df

,train_mse_errors,val_mse_errors,train_mae_errors,val_mae_errors,train_corrs,train_corr_pvals,validation_corr,validation_corr_pval
0,0.067559,0.469054,1.827186,4.914937,98.750650,0.0,89.008526,1.200906e-99
1,0.068395,0.442329,1.820982,4.852125,98.723898,0.0,91.032039,6.984970e-112
2,0.065270,0.505735,1.813709,4.870343,98.802618,0.0,88.465894,1.938489e-96
3,0.070137,0.495580,1.845628,5.004709,98.677877,0.0,89.728680,2.378498e-103
4,0.069142,0.565113,1.845183,5.176936,98.722119,0.0,85.976947,6.406394e-85
5,0.070254,0.447954,1.837370,4.851087,98.717402,0.0,88.603990,3.732097e-97
6,0.067307,0.396547,1.835443,4.500032,98.752498,0.0,91.667712,6.154616e-116
7,0.068440,0.552359,1.837464,5.406611,98.728358,0.0,87.353473,5.457404e-91
8,0.066760,0.489242,1.810191,4.936337,98.766592,0.0,89.499094,4.986167e-102
9,0.069163,0.477334,1.845142,4.692167,98.721272,0.0,88.398869,4.280078e-96


In [64]:
## For K = 10

In [65]:
model = linear_model.BayesianRidge()
predictAffinityWithModel(model)

100%|██████████| 10/10 [00:00<00:00, 69.77it/s]


(   train_mse_error  train_mse_std  val_mse_error  val_mse_std  \
 0           1.7538         0.0136         1.7901       0.1281   
 
    train_mae_error  train_mae_std  val_mae_error  val_mae_std  train_corr  \
 0           10.131         0.0436        10.2267       0.3822      0.4916   
 
    train_corr_pval  validation_corr  validation_corr_pval  
 0              0.0           0.4772                   0.0  ,
    train_mse_errors  val_mse_errors  train_mae_errors  val_mae_errors  \
 0          1.757003        1.756543         10.130397       10.231082   
 1          1.746865        1.877006         10.123038       10.339209   
 2          1.756751        1.756783         10.155731        9.883432   
 3          1.737997        1.938557         10.065078       10.739856   
 4          1.762882        1.714857         10.171091        9.905500   
 5          1.784332        1.498729         10.212101        9.479030   
 6          1.759358        1.738988         10.160627       10.167

In [66]:
model = RandomForestRegressor(n_estimators=100,n_jobs=multiprocessing.cpu_count(),random_state=42)
predictAffinityWithModel(model)

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


(   train_mse_error  train_mse_std  val_mse_error  val_mse_std  \
 0           0.0682         0.0015         0.4841       0.0478   
 
    train_mae_error  train_mae_std  val_mae_error  val_mae_std  train_corr  \
 0           1.8318         0.0125         4.9205       0.2346      0.9874   
 
    train_corr_pval  validation_corr  validation_corr_pval  
 0              0.0           0.8897                   0.0  ,
    train_mse_errors  val_mse_errors  train_mae_errors  val_mae_errors  \
 0          0.067559        0.469054          1.827186        4.914937   
 1          0.068395        0.442329          1.820982        4.852125   
 2          0.065270        0.505735          1.813709        4.870343   
 3          0.070137        0.495580          1.845628        5.004709   
 4          0.069142        0.565113          1.845183        5.176936   
 5          0.070254        0.447954          1.837370        4.851087   
 6          0.067307        0.396547          1.835443        4.500

In [67]:
model = RandomForestRegressor(n_estimators=50,n_jobs=multiprocessing.cpu_count(),random_state=42,max_depth=15)
predictAffinityWithModel(model)

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]


(   train_mse_error  train_mse_std  val_mse_error  val_mse_std  \
 0           0.1102         0.0031         0.4965       0.0468   
 
    train_mae_error  train_mae_std  val_mae_error  val_mae_std  train_corr  \
 0            2.415         0.0429         5.0266       0.2235      0.9787   
 
    train_corr_pval  validation_corr  validation_corr_pval  
 0              0.0           0.8871                   0.0  ,
    train_mse_errors  val_mse_errors  train_mae_errors  val_mae_errors  \
 0          0.105495        0.488541          2.348009        5.046114   
 1          0.107847        0.437135          2.351157        4.893935   
 2          0.107709        0.528474          2.411048        4.992903   
 3          0.111220        0.517198          2.412659        5.107428   
 4          0.107776        0.566480          2.405104        5.271641   
 5          0.113054        0.466815          2.433438        4.961250   
 6          0.115370        0.408429          2.486269        4.595

In [68]:
model = RandomForestRegressor(n_estimators=30,n_jobs=multiprocessing.cpu_count(),random_state=42,max_depth=12)
predictAffinityWithModel(model)

100%|██████████| 10/10 [00:02<00:00,  3.51it/s]


(   train_mse_error  train_mse_std  val_mse_error  val_mse_std  \
 0           0.1797         0.0062         0.5417       0.0451   
 
    train_mae_error  train_mae_std  val_mae_error  val_mae_std  train_corr  \
 0            3.099         0.0559         5.3262       0.2043       0.964   
 
    train_corr_pval  validation_corr  validation_corr_pval  
 0              0.0           0.8762                   0.0  ,
    train_mse_errors  val_mse_errors  train_mae_errors  val_mae_errors  \
 0          0.176487        0.520168          3.044406        5.259202   
 1          0.173485        0.483121          3.029979        5.242784   
 2          0.179656        0.561943          3.129318        5.257106   
 3          0.179908        0.569328          3.090562        5.394189   
 4          0.167011        0.587457          2.995861        5.478064   
 5          0.191120        0.539383          3.181767        5.325848   
 6          0.184822        0.449026          3.155295        4.939

In [ ]:
model = RandomForestRegressor(n_estimators=30,n_jobs=multiprocessing.cpu_count(),random_state=42,max_depth=10)
predictAffinityWithModel(model)

In [ ]:
model = RandomForestRegressor(n_estimators=15,n_jobs=multiprocessing.cpu_count(),random_state=42,max_depth=10)
predictAffinityWithModel(model)

In [ ]:
np.min([i.tree_.max_depth for i in model.estimators_])